<a href="https://colab.research.google.com/github/aiscience-22/UA_War/blob/machine_learning_0.01/Twitter_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
columns = ['date', 'text', 'weight']
target = ['negative']

In [22]:
# we will need to use pyspark to connect to our joined_v001.csv file in AWS S3
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [6]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-08-24 19:47:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  3.41MB/s    in 0.3s    

2022-08-24 19:47:41 (3.41 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
# Add drivers to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TwitterEnsemble").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [23]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://jessehernandez70test1.s3.us-west-1.amazonaws.com/joined_v001.csv"

spark.sparkContext.addFile(url)
joined_data_df = spark.read.csv(SparkFiles.get("joined_v001.csv"), sep=",", header=True, inferSchema=True)


# Show DataFrame
joined_data_df.show()

+----------+--------------------+-----------+-----+------+
|      date|                text|   negative|event|weight|
+----------+--------------------+-----------+-----+------+
|2022-08-01|the conflict is b...|0.602711678| null|  null|
|2022-08-01|remember when sma...|0.212250158| null|  null|
|2022-08-01|hi  you have to u...|0.490129501| null|  null|
|2022-08-01|the world is in d...|0.876022816| null|  null|
|2022-08-01|will the imposed ...|0.044509333| null|  null|
|2022-08-01|owner of ukraines...| 0.83338201| null|  null|
|2022-08-01|animal shelter do...|0.013312108| null|  null|
|2022-08-01|    eb5f211900p4acc7|0.254985869| null|  null|
|2022-08-01|why  relations li...|0.409764558| null|  null|
|2022-08-01|crypto tattoos to...|0.005395876| null|  null|
|2022-08-01|its happening on ...|0.061495773| null|  null|
|2022-08-01|will certainly lo...|0.881493032| null|  null|
|2022-08-01|serbian vucic is ...|0.974542975| null|  null|
|2022-08-01|animal shelter do...|0.013312108| null|  nul

In [26]:
# Show the end of DataFrame
joined_data_df.tail(10) 

[Row(date='2022-08-12', text='would a president pence pardon an impeached trump  via', negative=None, event='\xa0Battles in eastern Ukraines Donetsk region continued with Russian forces conducting ground attacks in different locations including around the city of Bakhmut east of Siversk and northwest of the city of Donetsk', weight=0.5),
 Row(date='2022-08-12', text='conservatives say trump is not the nomineeso who is', negative=None, event='\xa0Battles in eastern Ukraines Donetsk region continued with Russian forces conducting ground attacks in different locations including around the city of Bakhmut east of Siversk and northwest of the city of Donetsk', weight=0.5),
 Row(date='2022-08-12', text='genioux fact  and their governments have become the true enemies of the fabulous and people predators as hyper mistaken as aggressive', negative=None, event='\xa0Battles in eastern Ukraines Donetsk region continued with Russian forces conducting ground attacks in different locations including

In [27]:
# convert spark dataframe to pandas

import pandas as pd
joined_data_df = joined_data_df.toPandas()
print(joined_data_df)




               date                                               text  \
0        2022-08-01  the conflict is being cast in binaries making ...   
1        2022-08-01  remember when smashed into seven fragments and...   
2        2022-08-01  hi  you have to understand that we are billion...   
3        2022-08-01  the world is in dire straits as it is not equi...   
4        2022-08-01  will the imposed on cause a significant shift ...   
...             ...                                                ...   
1048570  2022-08-12  conservatives say trump is not the nomineeso w...   
1048571  2022-08-12  s treason is what we get for letting  if youre...   
1048572  2022-08-12  trump and saudi arabia looks like theyve paid ...   
1048573  2022-08-12  i pray for the ukrainian people i want to be s...   
1048574  2022-08-12  refused to show his taxes that should have bee...   

         negative                                              event  weight  
0        0.602712               

In [28]:
# convert text to text lenght
joined_data_df['text_length']  = joined_data_df['text'].str.len()

joined_data_df.head()

,date,text,negative,event,weight,text_length
0,2022-08-01,the conflict is being cast in binaries making ...,0.602712,None,NaN,73
1,2022-08-01,remember when smashed into seven fragments and...,0.212250,None,NaN,108
2,2022-08-01,hi you have to understand that we are billion...,0.490130,None,NaN,112
3,2022-08-01,the world is in dire straits as it is not equi...,0.876023,None,NaN,130
4,2022-08-01,will the imposed on cause a significant shift ...,0.044509,None,NaN,128


In [29]:
# Drop the columns with text values
joined_data_df.drop("text", axis=1, inplace=True)
joined_data_df.drop("event", axis=1, inplace=True)

joined_data_df.head()

,date,negative,weight,text_length
0,2022-08-01,0.602712,NaN,73
1,2022-08-01,0.212250,NaN,108
2,2022-08-01,0.490130,NaN,112
3,2022-08-01,0.876023,NaN,130
4,2022-08-01,0.044509,NaN,128


In [30]:
# convert datetime to numerical

# importing the datetime module
import datetime as dt
 
joined_data_df['date'] = pd.to_datetime(joined_data_df['date'])
# joined_data_df['date'] = joined_data_df['date'].astype(np.int64)

joined_data_df['date'] = joined_data_df['date'].map(dt.datetime.toordinal)

In [31]:
joined_data_df.head()

,date,negative,weight,text_length
0,738368,0.602712,NaN,73
1,738368,0.212250,NaN,108
2,738368,0.490130,NaN,112
3,738368,0.876023,NaN,130
4,738368,0.044509,NaN,128


In [33]:
joined_data_df.tail()

,date,negative,weight,text_length
1048570,738379,NaN,0.5,51
1048571,738379,NaN,0.5,212
1048572,738379,NaN,0.5,224
1048573,738379,NaN,0.5,200
1048574,738379,NaN,0.5,260


In [19]:
# Drop the null rows
# joined_data_df = joined_data_df.dropna()

In [32]:
# get the shape of the new DataFrame
print(f"{joined_data_df.shape[0]} rows and {joined_data_df.shape[1]} columns")

1048575 rows and 4 columns


In [34]:
joined_data_df.head()

,date,negative,weight,text_length
0,738368,0.602712,NaN,73
1,738368,0.212250,NaN,108
2,738368,0.490130,NaN,112
3,738368,0.876023,NaN,130
4,738368,0.044509,NaN,128


# Split the Data into Training and Testing

In [36]:
# Create our features
X = joined_data_df.get_dummies(joined_data_df.drop(columns='negative'))

# Create our target
y = joined_data_df['negative']

AttributeError: ignored

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

Train the model using the training data.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score
Note: Use a random state of 1 for each algorithm to ensure consistency between tests

# Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators =100, random_state=1).fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
priority_features = pd.Series(data=rf_model.feature_importances_,index=X.columns)
priority_features.sort_values(ascending=False,inplace=True)
print(priority_features)

# Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec_model = EasyEnsembleClassifier(n_estimators =100, random_state=1).fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = eec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))